### Import packages

In [30]:
import pandas as pd
import numpy as np
import mlforecast
import lightgbm as lgb
from matplotlib import pyplot as plt
import optuna
import plotly_express as px
import seaborn as sns
import os
import gc
import warnings
import statsmodels.api as sm

### Import and analyse main datasets

#### Train Dataset

In [31]:
train = pd.read_parquet("../data/store_sales/train.parquet")
print(train.info())
print(train.describe())
display(train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
None
                 id     store_nbr         sales   onpromotion
count  3.000888e+06  3.000888e+06  3.000888e+06  3.000888e+06
mean   1.500444e+06  2.750000e+01  3.577757e+02  2.602770e+00
std    8.662819e+05  1.558579e+01  1.101998e+03  1.221888e+01
min    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
25%    7.502218e+05  1.400000e+01  0.000000e+00  0.000000e+00
50%    1.500444e+06  2.750000e+01  1.100000e+01  0.000000e+00
75%    2.250665e+06  4.100000e+01  1.958473e+02  0.000000e+00
max    3.000887e+06  5.400000e+01  1.247170e+05  7.410000e+02


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


#### Test Dataset

In [32]:
test = pd.read_parquet("../data/store_sales/test.parquet")
print(test.info())
print(test.describe())
display(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB
None
                 id     store_nbr   onpromotion
count  2.851200e+04  28512.000000  28512.000000
mean   3.015144e+06     27.500000      6.965383
std    8.230850e+03     15.586057     20.683952
min    3.000888e+06      1.000000      0.000000
25%    3.008016e+06     14.000000      0.000000
50%    3.015144e+06     27.500000      0.000000
75%    3.022271e+06     41.000000      6.000000
max    3.029399e+06     54.000000    646.000000


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


### Change date column to datetime

In [33]:
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)

### Analysis of Sales in different time scales

#### Daily sales over time as a bar plot

In [34]:
px.bar(train.groupby("date").agg({"sales": "sum"}).reset_index(), x="date", y="sales", title="Sales over time")

#### Daily sales over time as line plot

In [35]:
px.line(train.groupby("date").agg({"sales": "sum"}).reset_index(), x="date", y="sales", title="Sales over time")

#### Weekly sales per store

In [36]:
weekly_sales = train.set_index("date").groupby("store_nbr").resample("W").agg({"sales": "sum"}).reset_index()
px.line(weekly_sales, x="date", y="sales", color="store_nbr", title="Weekly sales per store")

#### Monthly sales per store

In [39]:
monthly_sales = train.set_index("date").groupby("store_nbr").resample("ME").agg({"sales": "sum"}).reset_index()
px.line(monthly_sales, x="date", y="sales", color="store_nbr", title="Monthly sales per store")

#### Yearly sales summed up by store

In [43]:
yearly_sales = train.set_index("date").groupby("store_nbr").resample("YE").agg({"sales": "sum"}).reset_index()
px.bar(yearly_sales, x="date", y="sales", title="Yearly sales per store")

### Info about stores dataset

In [18]:
stores = pd.read_parquet("../data/store_sales/stores.parquet")
print(stores.info())
print(stores.describe())
display(stores.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None
       store_nbr    cluster
count  54.000000  54.000000
mean   27.500000   8.481481
std    15.732133   4.693395
min     1.000000   1.000000
25%    14.250000   4.000000
50%    27.500000   8.500000
75%    40.750000  13.000000
max    54.000000  17.000000


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Merge information on stores with train data

In [45]:
merged_train = pd.merge(train, stores, on="store_nbr")
print(merged_train.isnull().sum())
print(merged_train.info())
display(merged_train)

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
city           0
state          0
type           0
cluster        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
 6   city         object        
 7   state        object        
 8   type         object        
 9   cluster      int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 228.9+ MB
None


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


#### Bar plot of total sales per city

In [47]:
px.bar(merged_train.groupby("city").agg({"sales": "sum"}).reset_index(), x="city", y="sales", title="Sales per city")

### Analysis of transactions per store per day

In [131]:
transactions = pd.read_parquet("../data/store_sales/transactions.parquet").sort_values(["store_nbr", "date"])
transactions["date"] = pd.to_datetime(transactions.date)
print(transactions["transactions"].max)
print(transactions.info())
display(transactions)

<bound method Series.max of 1        2111
47       1833
93       1863
139      1509
185       520
         ... 
83271     768
83325     903
83379    1054
83433     818
83487     802
Name: transactions, Length: 83488, dtype: int64>
<class 'pandas.core.frame.DataFrame'>
Index: 83488 entries, 1 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.5 MB
None


,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520
...,...,...,...
83271,2017-08-11,54,768
83325,2017-08-12,54,903
83379,2017-08-13,54,1054
83433,2017-08-14,54,818


#### Total transactions per store

In [132]:
px.bar(transactions.groupby("store_nbr").agg({"transactions": "sum"}).reset_index(), x="store_nbr", y="transactions", title="Transactions per store")

#### Transactions per store per month

In [133]:
transactions_temp = transactions.set_index("date").groupby("store_nbr").resample("ME").agg({"transactions": "sum"}).reset_index()
px.line(transactions_temp, x="date", y="transactions", title="Transactions per store", color="store_nbr")

### Merging transactions with train

In [134]:

transactions_merged_train = pd.merge(merged_train.sort_values(["date", "store_nbr"]), transactions, on=["date", "store_nbr"], how = "left")
print(transactions_merged_train.isnull().sum())
print(transactions_merged_train.info())

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type                 0
cluster              0
transactions    245784
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   int64         
 6   city          object        
 7   state         object        
 8   type          object        
 9   cluster       int64         
 10  transactions  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 251.8+ MB
None


#### Filling days with 0 total sales with 0 transactions

In [138]:
transactions_merged_train['total_sales'] = transactions_merged_train.groupby(['date', 'store_nbr'])['sales'].transform('sum')
transactions_merged_train.loc[transactions_merged_train['total_sales'] == 0, 'transactions'] = transactions_merged_train.loc[transactions_merged_train['total_sales'] == 0, 'transactions'].fillna(0)
print(f"NaN Values in Column transactions: {transactions_merged_train['transactions'].isnull().sum()}")

NaN Values in Column transactions: 3894


#### Calculating ratio between total sales and transactions on a day

In [139]:
transactions_merged_train['sales_transactions_ratio'] = transactions_merged_train['total_sales'] / transactions_merged_train['transactions']
median_value = transactions_merged_train['sales_transactions_ratio'].median()
print(f"Median ratio between sales and transactions: {median_value}")

Median ratio between sales and transactions: 7.156389264410902


#### Dividing total sales on a day with NaN transaction values by median ratio

In [140]:
transactions_merged_train['transactions'] = transactions_merged_train['transactions'].fillna(transactions_merged_train['total_sales'] / median_value)
transactions_merged_train['transactions'] = transactions_merged_train['transactions'].astype(int)
print(f"NaN Values in Column transactions: {transactions_merged_train['transactions'].isnull().sum()}")

NaN Values in Column transactions: 0


#### Dropping helper columns

In [141]:
transactions_merged_train = transactions_merged_train.drop(columns=["total_sales", "sales_transactions_ratio"])

### Analysis of USD to EURO Exchange rate dataset 

In [145]:
currency_parquet = pd.read_parquet('../data/forex.parquet')
currency_parquet["date"] = pd.to_datetime(currency_parquet["date"])
print(currency_parquet.info())
display(currency_parquet)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453035 entries, 0 to 1453034
Data columns (total 7 columns):
 #   Column    Non-Null Count    Dtype         
---  ------    --------------    -----         
 0   slug      1453035 non-null  object        
 1   date      1453035 non-null  datetime64[ns]
 2   open      1453035 non-null  float64       
 3   high      1453035 non-null  float64       
 4   low       1453035 non-null  float64       
 5   close     1453035 non-null  float64       
 6   currency  1453035 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 77.6+ MB
None


,slug,date,open,high,low,close,currency
0,GBP/EGP,2001-04-10,5.580900,5.594700,5.594700,5.594700,EGP
1,GBP/EGP,2001-06-04,5.475170,5.493900,5.493900,5.493900,EGP
2,GBP/EGP,2001-08-01,5.679900,5.654300,5.654300,5.654300,EGP
3,GBP/EGP,2002-07-29,7.217000,7.217000,7.217000,7.217000,EGP
4,GBP/EGP,2003-01-02,7.424290,7.389900,7.389900,7.389900,EGP
...,...,...,...,...,...,...,...
1453030,USD/MXN,2021-08-24,20.317129,20.360830,20.238400,20.317949,MXN
1453031,USD/MXN,2021-08-25,20.202400,20.347589,20.160601,20.206100,MXN
1453032,USD/MXN,2021-08-26,20.228861,20.384501,20.211901,20.216101,MXN
1453033,USD/MXN,2021-08-27,20.369301,20.423140,20.191200,20.366501,MXN


#### Selecting USD as it is the currency of Ecuador

In [146]:
us_currency = currency_parquet[currency_parquet['currency'] == 'USD']
print(us_currency.shape)

(46937, 7)


#### Selecting dates relevant for train dataset

In [147]:
filtered_us_currency = us_currency[(us_currency['date'] >= transactions_merged_train["date"].min()) & (us_currency['date'] <= transactions_merged_train['date'].max())].reset_index(drop=True)
print(filtered_us_currency.shape)

(13247, 7)


#### Selecting rows with exchange rate between EUR and USD

In [148]:
final_us_currency = filtered_us_currency[filtered_us_currency['slug'] == 'EUR/USD'].drop(['slug', "currency"], axis=1).reset_index(drop=True)
final_us_currency = final_us_currency.rename(columns={"open": "slug_eur_usd_open", "high": "slug_eur_usd_high", "low": "slug_eur_usd_low", "close": "slug_eur_usd_close"})
display(final_us_currency)

,date,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close
0,2013-01-01,1.320132,1.322401,1.319592,1.320132
1,2013-01-02,1.319784,1.329805,1.319435,1.319784
2,2013-01-03,1.318392,1.318913,1.308900,1.318218
3,2013-01-04,1.303101,1.306380,1.300052,1.303271
4,2013-01-07,1.306848,1.309929,1.302060,1.306745
...,...,...,...,...,...
1199,2017-08-09,1.175337,1.176028,1.169235,1.175461
1200,2017-08-10,1.176471,1.176471,1.170467,1.176152
1201,2017-08-11,1.176997,1.183180,1.174922,1.177024
1202,2017-08-14,1.182634,1.183999,1.177329,1.182550


#### Merging with train dataset

In [150]:
exchange_merged_train = transactions_merged_train.merge(final_us_currency, how="left", on="date")
print(exchange_merged_train.isnull().sum())

id                         0
date                       0
store_nbr                  0
family                     0
sales                      0
onpromotion                0
city                       0
state                      0
type                       0
cluster                    0
transactions               0
slug_eur_usd_open     860706
slug_eur_usd_high     860706
slug_eur_usd_low      860706
slug_eur_usd_close    860706
dtype: int64


#### Filling missing values on weekends using forward fill

In [151]:
exchange_merged_train = exchange_merged_train.ffill()
print(exchange_merged_train.isnull().sum())

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type                  0
cluster               0
transactions          0
slug_eur_usd_open     0
slug_eur_usd_high     0
slug_eur_usd_low      0
slug_eur_usd_close    0
dtype: int64


### Analysis of General Inflation in Ecuador per year

In [156]:
global_inflation_df = pd.read_csv('../data/global_inflation_data.csv')
print(global_inflation_df.info())
display(global_inflation_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    196 non-null    object 
 1   indicator_name  196 non-null    object 
 2   1980            140 non-null    float64
 3   1981            144 non-null    float64
 4   1982            145 non-null    float64
 5   1983            145 non-null    float64
 6   1984            145 non-null    float64
 7   1985            145 non-null    float64
 8   1986            145 non-null    float64
 9   1987            147 non-null    float64
 10  1988            147 non-null    float64
 11  1989            147 non-null    float64
 12  1990            150 non-null    float64
 13  1991            155 non-null    float64
 14  1992            158 non-null    float64
 15  1993            169 non-null    float64
 16  1994            171 non-null    float64
 17  1995            172 non-null    flo

,country_name,indicator_name,1980,1981,1982,1983,1984,1985,1986,1987,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,Annual average inflation (consumer prices) rate,13.4,22.2,18.2,15.9,20.4,8.7,-2.1,18.4,...,-0.66,4.38,4.98,0.63,2.3,5.44,5.06,13.71,9.1,NaN
1,Albania,Annual average inflation (consumer prices) rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.90,1.30,2.00,2.00,1.4,1.60,2.00,6.70,4.8,4.0
2,Algeria,Annual average inflation (consumer prices) rate,9.7,14.6,6.6,7.8,6.3,10.4,14.0,5.9,...,4.80,6.40,5.60,4.30,2.0,2.40,7.20,9.30,9.0,6.8
3,Andorra,Annual average inflation (consumer prices) rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.10,-0.40,2.60,1.00,0.5,0.10,1.70,6.20,5.2,3.5
4,Angola,Annual average inflation (consumer prices) rate,46.7,1.4,1.8,1.8,1.8,1.8,1.8,1.8,...,9.20,30.70,29.80,19.60,17.1,22.30,25.80,21.40,13.1,22.3


### Selecting only values for Ecuador

In [157]:
global_inflation_df = global_inflation_df[global_inflation_df['country_name'] == 'Ecuador']

#### Fixing Dataset by switching index/headers

In [158]:
display(global_inflation_df.head())
global_inflation_df = global_inflation_df.transpose()
global_inflation_df = global_inflation_df.iloc[2:]
global_inflation_df = global_inflation_df.reset_index()
global_inflation_df.columns = ['date', 'inflation_ecuador']
global_inflation_df["date"] = pd.to_datetime(global_inflation_df["date"])
display(global_inflation_df.head())

,country_name,indicator_name,1980,1981,1982,1983,1984,1985,1986,1987,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
50,Ecuador,Annual average inflation (consumer prices) rate,13.0,16.4,16.3,48.4,31.2,28.0,23.0,29.5,...,4.0,1.7,0.4,-0.2,0.3,-0.3,0.1,3.5,2.3,1.8


,date,inflation_ecuador
0,1980-01-01,13.0
1,1981-01-01,16.4
2,1982-01-01,16.3
3,1983-01-01,48.4
4,1984-01-01,31.2


#### Selecting only relevant date range

In [161]:
global_inflation_df = global_inflation_df[(global_inflation_df['date'] >= exchange_merged_train["date"].min()) & (global_inflation_df['date'] <= exchange_merged_train['date'].max())] 
global_inflation_df = global_inflation_df.reset_index(drop=True)
display(global_inflation_df.head()) 

,date,inflation_ecuador
0,2013-01-01,2.7
1,2014-01-01,3.6
2,2015-01-01,4.0
3,2016-01-01,1.7
4,2017-01-01,0.4


#### Merging dataframe with train

In [162]:
ecuador_inflation_merged_train = exchange_merged_train.merge(global_inflation_df, how="left", on="date")
print(ecuador_inflation_merged_train.isnull().sum())
print(ecuador_inflation_merged_train.info())

id                          0
date                        0
store_nbr                   0
family                      0
sales                       0
onpromotion                 0
city                        0
state                       0
type                        0
cluster                     0
transactions                0
slug_eur_usd_open           0
slug_eur_usd_high           0
slug_eur_usd_low            0
slug_eur_usd_close          0
inflation_ecuador     2991978
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   id                  int64         
 1   date                datetime64[ns]
 2   store_nbr           int64         
 3   family              object        
 4   sales               float64       
 5   onpromotion         int64         
 6   city                object        
 7   state               object   

#### Filling all NaN values in column with forward fill

In [164]:
ecuador_inflation_merged_train = ecuador_inflation_merged_train.ffill()
print(f"NaN values in column inflation_ecuador: {ecuador_inflation_merged_train['inflation_ecuador'].isnull().sum()}")

NaN values in column inflation_ecuador: 0


### Analysis of USD inflation per month

In [168]:
usd_inflation_df = pd.read_csv('../data/inflation_per_month.csv')
usd_inflation_df["date"] = pd.to_datetime(usd_inflation_df["DATE"])
usd_inflation_df = usd_inflation_df.drop(["DATE"], axis=1)
usd_inflation_df = usd_inflation_df.rename(columns={"CPIUFDNS_PC1": "inflation_usd"})
print(usd_inflation_df.info())
display(usd_inflation_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   inflation_usd  1310 non-null   float64       
 1   date           1310 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 20.6 KB
None


,inflation_usd,date
0,6.18557,1914-01-01
1,4.16667,1914-02-01
2,2.08333,1914-03-01
3,-1.03093,1914-04-01
4,1.04167,1914-05-01


#### Selecting relevant date range

In [169]:
usd_inflation_df = usd_inflation_df[(usd_inflation_df['date'] >= ecuador_inflation_merged_train["date"].min()) & (usd_inflation_df['date'] <= ecuador_inflation_merged_train['date'].max())]
usd_inflation_df = usd_inflation_df.reset_index(drop=True)
print(usd_inflation_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   inflation_usd  56 non-null     float64       
 1   date           56 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.0 KB
None


#### Merging dataframe with train

In [170]:
usd_inflation_merged_train = ecuador_inflation_merged_train.merge(usd_inflation_df, how="left", on="date")
print(usd_inflation_merged_train.isnull().sum())
print(usd_inflation_merged_train.info())

id                          0
date                        0
store_nbr                   0
family                      0
sales                       0
onpromotion                 0
city                        0
state                       0
type                        0
cluster                     0
transactions                0
slug_eur_usd_open           0
slug_eur_usd_high           0
slug_eur_usd_low            0
slug_eur_usd_close          0
inflation_ecuador           0
inflation_usd         2901096
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   id                  int64         
 1   date                datetime64[ns]
 2   store_nbr           int64         
 3   family              object        
 4   sales               float64       
 5   onpromotion         int64         
 6   city                object        
 7  

#### Filling NaN values using forward fill

In [172]:
usd_inflation_merged_train = usd_inflation_merged_train.ffill()
print(f"NaN values in column inflation_usd: {usd_inflation_merged_train['inflation_usd'].isnull().sum()}")

NaN values in column inflation_usd: 0


What to do next:
- Add oil price to dataframe
- Add holidays to dataframe
- automatically detect when a store got opened up and filter out rows with no value
- get correlation visualization between all features
- One hot encode categorical values
- add way automatic way of training different clusters etc
- Create new unique_id feature
- drop store_nbr for training
- Future Leakage Fix einbauen
- Analyse verschiedener Modelle